<a href="https://colab.research.google.com/github/OnePassio/CE888/blob/master/Project/Experiment_2_Detect_Covariate_Shift_in_microsoft_malware_prediction_full_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Library

In [0]:
# Initial Library
# data analysis
import pandas as pd
import numpy as np
import random as rand

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# ignore warning message
import warnings
warnings.filterwarnings('ignore')

# machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 1.Load Data from Kaggle
Source Data: https://www.kaggle.com/c/microsoft-malware-prediction/

!kaggle competitions download -c microsoft-malware-prediction

In [2]:
# load data from kaggle
import os
os.environ['KAGGLE_USERNAME'] = "thomastran89" # username from the json file
os.environ['KAGGLE_KEY'] = "6d1907f735446c9c560bb4371074fecd" # key from the json file
!kaggle competitions download -c microsoft-malware-prediction # api copied from kaggle
# Unzip the data
!sample_submission.csv.zip
!unzip test.csv.zip
!unzip train.csv.zip
# Check files after unzip
!ls

 98% 756M/768M [00:28<00:00, 33.9MB/s]
100% 768M/768M [00:28<00:00, 28.0MB/s]
100% 671M/672M [00:26<00:00, 17.8MB/s]
100% 672M/672M [00:26<00:00, 27.1MB/s]
 91% 122M/134M [00:05<00:00, 19.7MB/s]
100% 134M/134M [00:05<00:00, 26.2MB/s]
/bin/bash: sample_submission.csv.zip: command not found
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               
sample_data		   test.csv	 train.csv
sample_submission.csv.zip  test.csv.zip  train.csv.zip


In [5]:
# load partial data for test first
sample_size= 200000 # data too heavy and can not load by Google Colab -> Test on 1M first rows

# Load Train Data
#df_train = pd.read_csv("train.csv",skiprows=0, nrows=sample_size)
df_train = pd.read_csv("train.csv")

# Load Test Data
#df_test= pd.read_csv("test.csv",skiprows=0, nrows=sample_size)
df_test= pd.read_csv("test.csv")

df_train.head()
#df_test.head()


MachineIdentifier  ... HasDetections
0  0000028988387b115f69f31a3bf04f09  ...             0
1  000007535c3f730efa9ea0b7ef1bd645  ...             0
2  000007905a28d863f6d0d597892cd692  ...             0
3  00000b11598a75ea8ba1beea8459149f  ...             1
4  000014a5f00daa18e76b81417eeb99fc  ...             1

[5 rows x 83 columns]

# 2. Preprocessing Data

## 2.1. Remove some unused features to build model of dataset
Some column field data have a unique ID each row or have too much distinct values(>50) should be remove, because it will make hard for build ML model

In [6]:
# Get All Accept Field from dataset (Ignore the field with too much string values) to build ML model
MAX_VALUE_ACCEPT=50
listAcceptField_Train=[]
for x in list(df_train):
    count=df_train[x].nunique()
    #print("{} has unique value is:{}".format(x,count))
    if(count<MAX_VALUE_ACCEPT):
        listAcceptField_Train.append(x)

listAcceptField_Test=listAcceptField_Train.copy()
listAcceptField_Test.remove('HasDetections')

print("Accept Fields from dataset to build Model is:")
print(listAcceptField_Train)
print(listAcceptField_Test)

Accept Fields from dataset to build Model is:
['ProductName', 'IsBeta', 'RtpStateBitfield', 'IsSxsPassiveMode', 'AVProductsInstalled', 'AVProductsEnabled', 'HasTpm', 'OrganizationIdentifier', 'Platform', 'Processor', 'OsSuite', 'OsPlatformSubRelease', 'SkuEdition', 'IsProtected', 'AutoSampleOptIn', 'PuaMode', 'SMode', 'SmartScreen', 'Firewall', 'UacLuaenable', 'Census_MDC2FormFactor', 'Census_DeviceFamily', 'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier', 'Census_ProcessorClass', 'Census_PrimaryDiskTypeName', 'Census_HasOpticalDiskDrive', 'Census_PowerPlatformRoleName', 'Census_OSArchitecture', 'Census_OSBranch', 'Census_OSEdition', 'Census_OSSkuName', 'Census_OSInstallTypeName', 'Census_OSInstallLanguageIdentifier', 'Census_OSWUAutoUpdateOptionsName', 'Census_IsPortableOperatingSystem', 'Census_GenuineStateName', 'Census_ActivationChannel', 'Census_IsFlightingInternal', 'Census_IsFlightsDisabled', 'Census_FlightRing', 'Census_ThresholdOptIn', 'Census_IsSecureBoot

In [7]:
#cache
df_train_id=df_train['MachineIdentifier'].copy()
df_test_id=df_test['MachineIdentifier'].copy()
# Just keep acceptable features
df_train=df_train[listAcceptField_Train].copy();
df_test=df_test[listAcceptField_Test].copy();

df_test.head()

ProductName  IsBeta  ...  Wdft_IsGamer  Wdft_RegionIdentifier
0  win8defender       0  ...           0.0                    7.0
1  win8defender       0  ...           1.0                   12.0
2  win8defender       0  ...           1.0                   11.0
3  win8defender       0  ...           0.0                   10.0
4  win8defender       0  ...           1.0                    3.0

[5 rows x 50 columns]

### 2.2. Fill NaN missing value

In [8]:
## missing values on train.csv
for i in df_train.columns:
    if df_train[i].dtype == 'object':
      df_train[i] = df_train[i].fillna(df_train[i].mode().iloc[0])
    if (df_train[i].dtype == 'int' or df_train[i].dtype == 'float'):
      df_train[i] = df_train[i].fillna(np.mean(df_train[i]))

## missing values on test.csv
for i in df_test.columns:
    if df_test[i].dtype == 'object':
      df_test[i] = df_test[i].fillna(df_test[i].mode().iloc[0])
    if (df_test[i].dtype == 'int' or df_test[i].dtype == 'float'):
      df_test[i] = df_test[i].fillna(np.mean(df_test[i]))
      
df_test.head()

ProductName  IsBeta  ...  Wdft_IsGamer  Wdft_RegionIdentifier
0  win8defender       0  ...           0.0                    7.0
1  win8defender       0  ...           1.0                   12.0
2  win8defender       0  ...           1.0                   11.0
3  win8defender       0  ...           0.0                   10.0
4  win8defender       0  ...           1.0                    3.0

[5 rows x 50 columns]

## 2.3. label encoding

In [9]:
# encoder, alternative is convert to dummies
number = LabelEncoder()
for i in df_train.columns:
    if (df_train[i].dtype == 'object'):
      df_train[i] = number.fit_transform(df_train[i].astype('str'))
      df_train[i] = df_train[i].astype('object')

for i in df_test.columns:
    if (df_test[i].dtype == 'object'):
      df_test[i] = number.fit_transform(df_test[i].astype('str'))
      df_test[i] = df_test[i].astype('object')

df_test.head()

ProductName  IsBeta  ...  Wdft_IsGamer  Wdft_RegionIdentifier
0           4       0  ...           0.0                    7.0
1           4       0  ...           1.0                   12.0
2           4       0  ...           1.0                   11.0
3           4       0  ...           0.0                   10.0
4           4       0  ...           1.0                    3.0

[5 rows x 50 columns]

## 2.4. creating a new feature origin


In [10]:
df_train['origin'] = 0
df_test['origin'] = 1
df_test.head()

ProductName  IsBeta  ...  Wdft_RegionIdentifier  origin
0           4       0  ...                    7.0       1
1           4       0  ...                   12.0       1
2           4       0  ...                   11.0       1
3           4       0  ...                   10.0       1
4           4       0  ...                    3.0       1

[5 rows x 51 columns]

# 3. Detect Covariate Shift: Detect Dataset Shift feature

In [0]:
training = df_train.drop('HasDetections',axis=1) # droping target variable

## taking sample from training and test data
training = training.sample(100000, random_state=12)
testing = df_test.sample(100000, random_state=11)
## combining random samples
combi_data = training.append(testing)
y = combi_data['origin']
combi_data.drop('origin',axis=1,inplace=True)

In [12]:
## modelling
from sklearn.model_selection import cross_val_score

ROC_AUC_THRESHOLD=0.8
model = RandomForestClassifier(n_estimators = 100, max_depth = 5,min_samples_leaf = 5)
drop_list = []
print("Features have roc_auc greater than {} are:".format(ROC_AUC_THRESHOLD))
for i in combi_data.columns:
  score = cross_val_score(model,pd.DataFrame(combi_data[i]),y,cv=2,scoring='roc_auc')
  if (np.mean(score) > ROC_AUC_THRESHOLD):
    drop_list.append(i)
    print(i,np.mean(score))


Features have roc_auc greater than 0.8 are:
SMode 0.879474684392
Census_OSBranch 0.890804572868
Census_OSEdition 0.994298115344
Census_OSSkuName 0.814374224264
Census_IsFlightingInternal 0.9873739454999999
Census_FlightRing 0.985289748352
Census_ThresholdOptIn 0.946219568392
Census_IsWIMBootEnabled 0.945936794884


In [13]:
print("Total Features of dataset is {}, dataset shift features are {}".format(len(list(combi_data)),len(drop_list)))
print(drop_list)
remain_features=[]
for x in list(combi_data):
  if(x not in drop_list):
    remain_features.append(x)
print("features have low drift value:")
print(remain_features)

Total Features of dataset is 50, dataset shift features are 8
['SMode', 'Census_OSBranch', 'Census_OSEdition', 'Census_OSSkuName', 'Census_IsFlightingInternal', 'Census_FlightRing', 'Census_ThresholdOptIn', 'Census_IsWIMBootEnabled']
features have low drift value:
['ProductName', 'IsBeta', 'RtpStateBitfield', 'IsSxsPassiveMode', 'AVProductsInstalled', 'AVProductsEnabled', 'HasTpm', 'OrganizationIdentifier', 'Platform', 'Processor', 'OsSuite', 'OsPlatformSubRelease', 'SkuEdition', 'IsProtected', 'AutoSampleOptIn', 'PuaMode', 'SmartScreen', 'Firewall', 'UacLuaenable', 'Census_MDC2FormFactor', 'Census_DeviceFamily', 'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier', 'Census_ProcessorClass', 'Census_PrimaryDiskTypeName', 'Census_HasOpticalDiskDrive', 'Census_PowerPlatformRoleName', 'Census_OSArchitecture', 'Census_OSInstallTypeName', 'Census_OSInstallLanguageIdentifier', 'Census_OSWUAutoUpdateOptionsName', 'Census_IsPortableOperatingSystem', 'Census_GenuineStateName', '

# 4. Build classifier prediction without features having a drift value greater

In [14]:

from sklearn.model_selection import train_test_split
#Get the values and run a classifier 
features= remain_features

# convert to dummies
X=df_train[features].copy()
y=df_train['HasDetections']

# 70% train, 30% test from dataset train.csv
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

X_train

ProductName  IsBeta  ...  Wdft_IsGamer  Wdft_RegionIdentifier
3667547           4       0  ...           0.0                   11.0
6327294           4       0  ...           0.0                    3.0
2550853           4       0  ...           0.0                   11.0
312458            4       0  ...           0.0                   11.0
1700297           4       0  ...           1.0                   11.0
...             ...     ...  ...           ...                    ...
3417834           4       0  ...           0.0                    5.0
8214018           4       0  ...           1.0                    4.0
4312712           4       0  ...           1.0                   11.0
7309649           4       0  ...           0.0                   12.0
2478997           4       0  ...           0.0                   10.0

[6245038 rows x 42 columns]

In [0]:

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score

# main Model
clf = ExtraTreesClassifier(n_estimators=1000, max_depth=4)
clf.fit(X_train,y_train)

# test with dummy model
#dummy_clf = DummyClassifier() # generates predictions by respecting the training set's class distribution
#dummy_clf.fit(X_train,y_train)

In [0]:
#scores = cross_val_score(clf, X_train, y_train, cv=10,scoring=make_scorer(accuracy_score))
#dummy_scores = cross_val_score(dummy_clf, X_train, y_train, cv=10, scoring=make_scorer(accuracy_score))

#Report the results of 10-Kfold stratified cross-validation of 2 method
#print("ACC: %0.2f +/- %0.2f" % (scores.mean(), scores.std()))
#print("Dummy ACC: %0.2f +/- %0.2f" % (dummy_scores.mean(), dummy_scores.std()))

4.1 Test On test.csv

In [0]:
# Load Test Data all
#df_test = pd.read_csv("test.csv",skiprows=0)
features= remain_features
testFeatures=df_test[features].copy()
pred = clf.predict(testFeatures)
columns = ['HasDetections']
sub = pd.DataFrame(data=pred,columns=columns)
sub['MachineIdentifier'] = df_test_id
sub = sub[['MachineIdentifier','HasDetections']]
sub.to_csv('predict_without_drift_value_greater.csv', index=False)

In [0]:
#download result
from google.colab import files
files.download('predict_without_drift_value_greater.csv')

In [0]:
sub